In [1]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import scipy.stats as stats

In [2]:
data = pd.read_csv('../synthetic_gmv_data_1.2.csv')

In [3]:
data.head()

,user_id,gmv,group_name
0,myo4ixol31,1428,test
1,myo4ixol31,1428,test
2,myo4ixol31,1071,test
3,myo4ixol31,1071,test
4,pkzf2889ww,351,test


In [4]:
test_data = data[data['group_name'] == 'test']
control_data = data[data['group_name'] != 'test']

In [5]:
agg_data_test = test_data.groupby('user_id').agg({'gmv': 'sum', 'user_id': 'count'}).rename(columns={'user_id': 'num_records'})
agg_data_control = control_data.groupby('user_id').agg({'gmv': 'sum', 'user_id': 'count'}).rename(columns={'user_id': 'num_records'})

In [7]:
agg_data_test.head(5)

,gmv,num_records
user_id,,
00074uxybk,3187,3
000plmykri,1695,5
0026yqk83k,1293,3
002ioy63br,3862,6
0032wrbd7c,765,3


In [8]:
control = agg_data_control.to_numpy()
test = agg_data_test.to_numpy()

In [9]:
control

array([[ 733,    1],
       [2933,    6],
       [1496,    4],
       ...,
       [2364,    4],
       [1235,    2],
       [3579,    5]], shape=(147291, 2))

In [23]:
t_stat, p_value = stats.ttest_ind(test, control, equal_var=False)

In [25]:
print(f'{t_stat:.3f} {p_value:.3f}')

2.360 0.018


In [10]:
def delta_method_avg_bill_ttest(gmv_data, transactions_data):
    """
    Расчет среднего чека и его стандартной ошибки с помощью дельта-метода
    """
    n = len(gmv_data)
    
    # Средние значения
    mean_gmv = np.mean(gmv_data)
    mean_transactions = np.mean(transactions_data)
    
    # Средний чек
    avg_bill = mean_gmv / mean_transactions
    
    # Дисперсии
    var_gmv = np.var(gmv_data, ddof=1)
    var_transactions = np.var(transactions_data, ddof=1)
    
    # Ковариация между GMV и количеством транзакций
    cov_gmv_transactions = np.cov(gmv_data, transactions_data, ddof=1)[0, 1]
    
    # Частные производные для дельта-метода f(x,y) = x/y
    partial_x = 1 / mean_transactions
    partial_y = -mean_gmv / (mean_transactions ** 2)
    
    # Дисперсия среднего чека по дельта-методу
    var_avg_bill = (partial_x ** 2) * (var_gmv / n) + \
                   (partial_y ** 2) * (var_transactions / n) + \
                   2 * partial_x * partial_y * (cov_gmv_transactions / n)
    
    # Стандартная ошибка
    se_avg_bill = np.sqrt(var_avg_bill)
    
    return avg_bill, se_avg_bill

# Извлекаем данные из агрегированных DataFrame
gmv_test = agg_data_test['gmv'].values
transactions_test = agg_data_test['num_records'].values

gmv_control = agg_data_control['gmv'].values
transactions_control = agg_data_control['num_records'].values

# Расчет среднего чека и стандартной ошибки для каждой группы
avg_bill_test, se_test = delta_method_avg_bill_ttest(gmv_test, transactions_test)
avg_bill_control, se_control = delta_method_avg_bill_ttest(gmv_control, transactions_control)

print(f"Средний чек в тестовой группе: {avg_bill_test:.3f} ± {se_test:.3f}")
print(f"Средний чек в контрольной группе: {avg_bill_control:.3f} ± {se_control:.3f}")

# Метод 1: Ручной расчет t-статистики для дельта-метода
diff_avg_bill = avg_bill_test - avg_bill_control
se_diff = np.sqrt(se_test**2 + se_control**2)

t_stat_delta = diff_avg_bill / se_diff
df_welch = (se_test**2 + se_control**2)**2 / (se_test**4/(len(gmv_test)-1) + se_control**4/(len(gmv_control)-1))
p_value_delta = 2 * (1 - stats.t.cdf(abs(t_stat_delta), df_welch))

print(f"\nДельта-метод:")
print(f"t-статистика: {t_stat_delta:.3f}")
print(f"p-value: {p_value_delta:.3f}")

# Метод 2: Использование stats.ttest_ind для сравнения GMV напрямую
# (это неправильно для среднего чека, но показано для сравнения)
t_stat_gmv, p_value_gmv = stats.ttest_ind(gmv_test, gmv_control, equal_var=False)
print(f"\nПрямое сравнение GMV (некорректно для среднего чека):")
print(f"t-статистика: {t_stat_gmv:.3f}")
print(f"p-value: {p_value_gmv:.3f}")

# Метод 3: Альтернативный подход - бутстрап для валидации дельта-метода
def bootstrap_avg_bill_diff(gmv1, trans1, gmv2, trans2, n_bootstrap=1000):
    """Бутстрап для оценки распределения разности средних чеков"""
    bootstrap_diffs = []
    
    for _ in range(n_bootstrap):
        # Бутстрап выборки
        idx1 = np.random.choice(len(gmv1), len(gmv1), replace=True)
        idx2 = np.random.choice(len(gmv2), len(gmv2), replace=True)
        
        # Средние чеки для бутстрап выборок
        avg_bill1 = np.mean(gmv1[idx1]) / np.mean(trans1[idx1])
        avg_bill2 = np.mean(gmv2[idx2]) / np.mean(trans2[idx2])
        
        bootstrap_diffs.append(avg_bill1 - avg_bill2)
    
    return np.array(bootstrap_diffs)

# Бутстрап для сравнения
bootstrap_diffs = bootstrap_avg_bill_diff(gmv_test, transactions_test, 
                                         gmv_control, transactions_control)

bootstrap_se = np.std(bootstrap_diffs)
bootstrap_t_stat = diff_avg_bill / bootstrap_se
bootstrap_p_value = 2 * (1 - stats.norm.cdf(abs(bootstrap_t_stat)))

print(f"\nБутстрап (для валидации):")
print(f"SE разности: {bootstrap_se:.3f} (дельта-метод: {se_diff:.3f})")
print(f"t-статистика: {bootstrap_t_stat:.3f}")
print(f"p-value: {bootstrap_p_value:.3f}")

Средний чек в тестовой группе: 704.206 ± 1.474
Средний чек в контрольной группе: 700.223 ± 0.845

Дельта-метод:
t-статистика: 2.344
p-value: 0.019

Прямое сравнение GMV (некорректно для среднего чека):
t-статистика: 2.360
p-value: 0.018

Бутстрап (для валидации):
SE разности: 1.762 (дельта-метод: 1.699)
t-статистика: 2.260
p-value: 0.024


In [11]:
def linearized_metric(X, Y):
    """
    Вычисляет линеаризованную метрику L(u) для каждого пользователя
    
    L(u) = X̄/Ȳ + (1/Ȳ)*X(u) - (X̄/Ȳ²)*Y(u)
    
    где:
    X(u) - значение числителя для пользователя u (GMV)
    Y(u) - значение знаменателя для пользователя u (количество транзакций)
    X̄ - среднее в группе по числителю
    Ȳ - среднее в группе по знаменателю
    """
    X_mean = np.mean(X)
    Y_mean = np.mean(Y)
    
    # Линеаризованная метрика для каждого пользователя
    L = X_mean/Y_mean + (1/Y_mean)*X - (X_mean/(Y_mean**2))*Y
    
    return L

# Вычисляем линеаризованную метрику для каждой группы
L_test = linearized_metric(gmv_test, transactions_test)
L_control = linearized_metric(gmv_control, transactions_control)

print(f"Размер тестовой группы: {len(L_test)}")
print(f"Размер контрольной группы: {len(L_control)}")
print(f"Среднее линеаризованной метрики в тестовой группе: {np.mean(L_test):.6f}")
print(f"Среднее линеаризованной метрики в контрольной группе: {np.mean(L_control):.6f}")

# Применяем обычный t-тест к линеаризованной метрике
t_stat_linearized, p_value_linearized = stats.ttest_ind(L_test, L_control, equal_var=False)

print(f"\nЛинеаризованная метрика:")
print(f"t-статистика: {t_stat_linearized:.3f}")
print(f"p-value: {p_value_linearized:.3f}")

# Сравнение всех методов
print(f"\n=== СРАВНЕНИЕ МЕТОДОВ ===")
print(f"Дельта-метод:        t={t_stat_delta:.3f}, p={p_value_delta:.3f}")
print(f"Линеаризованная:     t={t_stat_linearized:.3f}, p={p_value_linearized:.3f}")
print(f"Бутстрап:           t={bootstrap_t_stat:.3f}, p={bootstrap_p_value:.3f}")

# Проверяем, что среднее линеаризованной метрики равно среднему чеку
avg_bill_test_check = np.mean(gmv_test) / np.mean(transactions_test)
avg_bill_control_check = np.mean(gmv_control) / np.mean(transactions_control)

print(f"\n=== ПРОВЕРКА КОРРЕКТНОСТИ ===")
print(f"Средний чек тест (прямой расчет): {avg_bill_test_check:.6f}")
print(f"Средний чек тест (линеаризованная): {np.mean(L_test):.6f}")
print(f"Средний чек контроль (прямой расчет): {avg_bill_control_check:.6f}")
print(f"Средний чек контроль (линеаризованная): {np.mean(L_control):.6f}")

# Дополнительная статистика
print(f"\n=== ДОПОЛНИТЕЛЬНАЯ СТАТИСТИКА ===")
print(f"Стандартное отклонение линеаризованной метрики:")
print(f"  Тестовая группа: {np.std(L_test, ddof=1):.6f}")
print(f"  Контрольная группа: {np.std(L_control, ddof=1):.6f}")

Размер тестовой группы: 49100
Размер контрольной группы: 147291
Среднее линеаризованной метрики в тестовой группе: 704.205947
Среднее линеаризованной метрики в контрольной группе: 700.223472

Линеаризованная метрика:
t-статистика: 2.344
p-value: 0.019

=== СРАВНЕНИЕ МЕТОДОВ ===
Дельта-метод:        t=2.344, p=0.019
Линеаризованная:     t=2.344, p=0.019
Бутстрап:           t=2.260, p=0.024

=== ПРОВЕРКА КОРРЕКТНОСТИ ===
Средний чек тест (прямой расчет): 704.205947
Средний чек тест (линеаризованная): 704.205947
Средний чек контроль (прямой расчет): 700.223472
Средний чек контроль (линеаризованная): 700.223472

=== ДОПОЛНИТЕЛЬНАЯ СТАТИСТИКА ===
Стандартное отклонение линеаризованной метрики:
  Тестовая группа: 326.619692
  Контрольная группа: 324.344475
